In [1]:
import glob # libreria para extraer la rurta de los archivos
import re # libreria para manejos de expresiones regulares
import collections #### para poder contar los hash
import pandas as pd # libreria para manejo de bases datos
import numpy as np # libreria para manejo de vectores y arreglos
from nltk.corpus import stopwords, wordnet # importa las stop words y las palabras del ingles
from nltk.stem.porter import PorterStemmer # metodo para stemming
from nltk.stem.lancaster import LancasterStemmer # metodo para stemming
from nltk.stem import WordNetLemmatizer # metodo para lematizar

In [23]:
#Read Files
files_txt = glob.glob("/opt/datasets/mcda-pi1-20191/papers-txt/*.txt")

# instanciar la clase para lematizar
wordnet_lemmatizer = WordNetLemmatizer()
# llamamos al diccionario de stop words en ingles
sw = stopwords.words("english")

# Recorre los archivos para generar el bag of words
bag_of_words = {} # creacion de la estructura de datos (diccionario) para almacenar el bag of words
for file in files_txt:
    input_file = open(file,"r",encoding='utf-8') # abre los archivos
    texto = input_file.read() # carga el contenido del archivo
    texto = re.sub('[^A-Za-z0-9]+',' ',texto) # solo permanecen los elementos alfanumericos
    tokens = texto.split() # Realiza la tokenizacion
    #tokens = [w for w in tokens if wordnet.synsets(w)] # deja solo las palabras asociadas al ingles
    stemmer = PorterStemmer() # instancia una forma de stemming
    #stemmer2 = LancasterStemmer() # instancia una forma de stemming
    # aplica lematizacion, stemming, elimina de stop words y aplica reglas logicas para reducir la cantidad de tokens
    tokens = [wordnet_lemmatizer.lemmatize(stemmer.stem(w.lower()), pos="v") for w in tokens if (len(w)>1) and w.isalpha() and (w not in sw) and wordnet.synsets(w)] # Longitud mayor a 1
    #tokens = [w for w in tokens if wordnet.synsets(w)] # deja solo las palabras asociadas al ingles
    # realiza el conteo de la frecuencia de cada palabra para cada documento
    counter=collections.Counter(tokens)
    # para un diccionario almacenamos el key que es el nombre del documento y el value que es un diccionario con las key como palabras y los value como la frecuencia de la palabra
    bag_of_words[file] = dict(counter)


In [25]:
helper = {}
i = 0
for doc, words in bag_of_words.items():
    for word in words.keys():
        if word in helper:
            helper[word] += 1
        else:
            helper[word] = 1

for documento, words in bag_of_words.items():
    num_words = sum(bag_of_words[documento].values())
    for key, value in words.items():
        dic = {}
        dic['freq'] = value
        dic['freqR'] = value / num_words
        dic['tf'] = 1 + np.log(value)
        dic['idf'] = np.log(len(bag_of_words) / helper[key])
        dic['tf-idf'] = dic['tf'] * dic['idf']
        bag_of_words[documento][key] = dic


## Esta funcion ordena el bow

In [ ]:
bag_of_words[files_txt[0]]
#sort
import operator
sorted_x = sorted(bag_of_words[files_txt[0]].items(), key=lambda kv: kv[0])
sorted_x

In [ ]:
# Construccion de un DF con informacion de frecuencia de palabras por documento
keys, values = [], [] # creacion de listas vacias
for key, value in bag_of_words.items():
    keys.append(key) # adjunta cada palabra a una lista vacia
    values.append(value) # adjunta cada valor a una lista vacia
    
pdTest = pd.DataFrame(data=values, index=keys) # Genera el dataframe con los documentos, las palabras y las frecuencias 
pdTest[['aisss']]
pdTest[['aisss']].dropna(axis = 'rows')#.index[0]

In [13]:
if not wordnet.synsets('does'):
    print('Not an English Word')
else:
    print('yes')

yes


In [ ]:
'http' in sison

In [ ]:
len(sison)

In [ ]:
sison = [x for x in total if wordnet.synsets(x)]
sison

In [ ]:
total = sorted(pdTest.columns.tolist())

In [ ]:
len(total)

In [29]:
#query
def transformar_informacion (texto):
    texto = re.sub('[^A-Za-z0-9]+',' ',texto) # Caracteres especiales
    texto = texto.replace('í','i')
    tokens = texto.split()
    stemmer = PorterStemmer()
    stemmer2 = LancasterStemmer()
    tokens = [wordnet_lemmatizer.lemmatize(stemmer.stem(w.lower()), pos="v") for w in tokens if (len(w)>1) and w.isalpha() and w not in sw and wordnet.synsets(w)] # Longitud mayor a 1
    counter=collections.Counter(tokens)
    return dict(counter)

texto = "user friendly user"
query = transformar_informacion(texto)
query

{'friendli': 1, 'user': 2}

In [33]:
import operator
result_query = {}
for documento, words in bag_of_words.items():
    for key, value in words.items():
        if key in query:
            if documento in result_query:
                result_query[documento] += query[key] * value['tf']
            else:
                result_query[documento] = query[key] * value['tf']

                
resultado = sorted(result_query.items(), key=operator.itemgetter(1), reverse=True)
print("Son {} resultados.... {}".format(len(resultado), resultado))

Son 345 resultados.... [('/opt/datasets/mcda-pi1-20191/papers-txt/1504.03342.txt', 14.339221464982913), ('/opt/datasets/mcda-pi1-20191/papers-txt/1304.7480.txt', 13.549103091088817), ('/opt/datasets/mcda-pi1-20191/papers-txt/1312.0461.txt', 12.779736745099953), ('/opt/datasets/mcda-pi1-20191/papers-txt/1204.2606.txt', 12.656627474604603), ('/opt/datasets/mcda-pi1-20191/papers-txt/1512.00327.txt', 12.086850233838494), ('/opt/datasets/mcda-pi1-20191/papers-txt/1511.06030.txt', 12.047761041692553), ('/opt/datasets/mcda-pi1-20191/papers-txt/1403.5715.txt', 12.007892611890918), ('/opt/datasets/mcda-pi1-20191/papers-txt/1101.1477.txt', 11.939626599152001), ('/opt/datasets/mcda-pi1-20191/papers-txt/1508.06589.txt', 11.861633597686627), ('/opt/datasets/mcda-pi1-20191/papers-txt/1407.3556.txt', 11.704060527839234), ('/opt/datasets/mcda-pi1-20191/papers-txt/1204.6321.txt', 11.63497622707262), ('/opt/datasets/mcda-pi1-20191/papers-txt/1412.4171.txt', 11.591581091193483), ('/opt/datasets/mcda-pi1-

# Metricas

In [ ]:
#Contar documentos
len(bag_of_words)

In [ ]:
#Contar numero de palabras por documento despues de limpieza
x_doc = [] # documentos
y_doc = [] # num palabras unicas por doc
total_palabras_doc = [] # total palabras
for documento, words in bag_of_words.items():
    x_doc.append(documento)
    y_doc.append(len(words))
    numero_palabras = 0
    for word, values in words.items():
        numero_palabras += values['freq']
    total_palabras_doc.append(numero_palabras)

#Imprimir Solo un documento
print("El documento '{}' tiene {} palabras unicas despues del proceso de limpieza".format(x_doc[0][-15:], y_doc[0]))
print("El total de palabras contenido en el texto es de {}".format(total_palabras_doc[0]))

In [ ]:
palabras_raras = collections.OrderedDict()
palabras_tres_caracteres = collections.OrderedDict()
i = 0
for documento, words in bag_of_words.items():
    for word, values in words.items():
        if len(word) > 10 and values['freq'] < 2:
            palabras_raras[i] = word
            i+=1
        if bool(re.search(r'((\w)\2{2,})', word)):
            palabras_tres_caracteres[word] = values['freq']
palabras_raras

In [ ]:
palabras_tres_caracteres

# Cosas Varias

In [ ]:
#Buscar documentos con X palabra
#palabras
raras = []
#bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb
for documento, words in bag_of_words.items():
    if 'integrationsreihenfolg' in words:
        raras.append(documento)

#buscar palabrasde mas de 15 caracteres       
for palabra in palabras:
    if len(palabra) > 15:
        print(palabra)

In [ ]:
#/opt/datasets/mcda-pi1-20191/papers-txt/1501.02741.txt
bag_of_words['/opt/datasets/mcda-pi1-20191/papers-txt/1501.02741.txt']['user']

input_file = open('/opt/datasets/mcda-pi1-20191/papers-txt/1501.02741.txt',"r",encoding='utf-8')
texto = input_file.read()
#texto = re.sub('[^A-Za-z0-9]+',' ',texto) # Caracteres especiales
tokens = texto.split()
stemmer = PorterStemmer()
stemmer2 = LancasterStemmer()
tokens = [wordnet_lemmatizer.lemmatize(stemmer.stem(w.lower()), pos="v") for w in tokens if (len(w)>1) and w.isalpha() and w not in sw] # Longitud mayor a 1
counter=collections.Counter(tokens)
dict(counter)['einhäuser']

In [ ]:
#Traductor
def detect_language(text):

    languages_ratios = {}
    tokens = tokens = texto.split()
    tokens = [w.lower() for w in tokens]
    # Compute per language included in nltk number of unique stopwords appearing in analyzed text
    for language in stopwords.fileids():
        stopwords_set = set(stopwords.words(language))
        words_set = set(tokens)
        common_elements = words_set.intersection(stopwords_set)
        languages_ratios[language] = len(common_elements) # language "score"
    return max(languages_ratios, key=languages_ratios.get)


#----------------------------------------------------------------------
input_file = open(files_txt[0],"r",encoding='utf-8')
texto = input_file.read()
language = detect_language(texto)
language

In [ ]:
'einhäuser' in total

In [35]:
import ast